In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import time
import matplotlib.pyplot as plt

# Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
NUM_CLASSES = 4
EPOCHS = 20

# Dataset path
data_dir = "COVID-19_Radiography_Dataset"
classes = ["COVID", "Normal", "Viral Pneumonia", "Lung_Opacity"]

# Question 2: Split data with stratified holdout (70% train, 30% validation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Class mapping: {train_generator.class_indices}")

# Question 3-4: Import MobileNetV2 with ImageNet pretrained weights
base_model = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    alpha=0.75  # Using the x0.75 version as specified in the assignment
)
print("Base model loaded with ImageNet weights")

# Question 5: Replace the last layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Question 6-7: Display model architecture and parameters
print("\nModel Architecture:")
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}, Output Shape: {layer.output_shape}, Params: {layer.count_params()}")

total_params = model.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
non_trainable_params = total_params - trainable_params

print(f"\nTotal parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {non_trainable_params}")

# Question 8: Retrain using transfer learning with different freezing strategies
def train_and_evaluate(model, trainable_layers, strategy_name):
    # Reset model weights for fair comparison
    for layer in model.layers:
        if hasattr(layer, 'kernel_initializer') and hasattr(layer, 'bias_initializer'):
            layer.kernel.assign(layer.kernel_initializer(shape=layer.kernel.shape))
            layer.bias.assign(layer.bias_initializer(shape=layer.bias.shape))
    
    # Freeze/unfreeze layers according to strategy
    for layer in base_model.layers:
        layer.trainable = False
    
    # Unfreeze specific layers based on the strategy
    if trainable_layers > 0:
        for layer in base_model.layers[-trainable_layers:]:
            layer.trainable = True
    
    # Unfreeze FC layers
    for layer in model.layers[len(base_model.layers):]:
        layer.trainable = True
    
    # Recompile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train the model
    print(f"\nTraining with strategy: {strategy_name}")
    start_time = time.time()
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE
    )
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    
    # Evaluate the model
    validation_generator.reset()
    y_true = []
    y_pred = []
    
    for i in range(validation_generator.samples // BATCH_SIZE + 1):
        X, y = next(validation_generator)
        pred = model.predict(X)
        y_true.extend(np.argmax(y, axis=1))
        y_pred.extend(np.argmax(pred, axis=1))
        if len(y_true) >= validation_generator.samples:
            break
    
    # Trim to match validation samples exactly
    y_true = y_true[:validation_generator.samples]
    y_pred = y_pred[:validation_generator.samples]
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    sensitivity = recall  # Same as recall for multi-class
    
    print(f"Validation Metrics for {strategy_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Sensitivity: {sensitivity:.4f}")
    
    # Plot convergence curves
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy Curves - {strategy_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curves - {strategy_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'{strategy_name}_convergence.png')
    plt.show()
    
    return {
        'accuracy': accuracy,
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'sensitivity': sensitivity,
        'training_time': training_time,
        'history': history.history
    }

# Run the three strategies (Question 8)
results = {}

# Strategy 1: Last two convolutional layers + three FC layers
results['strategy1'] = train_and_evaluate(model, 2, "Last 2 Conv + FC layers")

# Strategy 2: Last convolutional layer + three FC layers
results['strategy2'] = train_and_evaluate(model, 1, "Last 1 Conv + FC layers")

# Strategy 3: Only the last three FC layers
results['strategy3'] = train_and_evaluate(model, 0, "Only FC layers")

# Question 9: Compare results
print("\nComparison of Results:")
metrics = ['accuracy', 'recall', 'precision', 'f1', 'sensitivity', 'training_time']
strategies = ['strategy1', 'strategy2', 'strategy3']
strategy_names = ["Last 2 Conv + FC layers", "Last 1 Conv + FC layers", "Only FC layers"]

# Create comparison table
comparison = {}
for metric in metrics:
    comparison[metric] = [results[s][metric] for s in strategies]

# Print comparison table
print("\n{:<20} {:<25} {:<25} {:<25}".format('Metric', *strategy_names))
print("-" * 95)
for metric in metrics:
    if metric == 'training_time':
        print("{:<20} {:<25.2f} {:<25.2f} {:<25.2f}".format(
            metric, *comparison[metric]
        ))
    else:
        print("{:<20} {:<25.4f} {:<25.4f} {:<25.4f}".format(
            metric, *comparison[metric]
        ))

# Save the final model
model.save('covid_xray_mobilenetv2_best.h5')
print("\nModel saved successfully.")

Found 29632 images belonging to 4 classes.
Found 12698 images belonging to 4 classes.
Training samples: 29632
Validation samples: 12698
Class mapping: {'COVID': 0, 'Normal': 1, 'Viral Pneumonia': 2, 'Lung_Opacity': 3}
5903360/5903360 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Base model loaded with ImageNet weights

Model Architecture:


AttributeError: 'InputLayer' object has no attribute 'output_shape'